In [1]:
from foundational_ssm.data_utils.dataset import TorchBrainDataset
from foundational_ssm.data_utils.samplers import GroupedRandomFixedWindowSampler
from foundational_ssm.data_utils.loaders import get_dataset_config
from torch.utils.data import DataLoader
from foundational_ssm.constants import DATA_ROOT
from omegaconf import OmegaConf

In [2]:
from line_profiler import LineProfiler
from foundational_ssm.data_utils.dataset import TorchBrainDataset

In [3]:
config_path = "/cs/student/projects1/ml/2024/mlaimon/foundational_ssm/configs/pretrain.yaml"
cfg = OmegaConf.load(config_path) 
train_dataset = TorchBrainDataset(
    root="../"+DATA_ROOT,
    config=get_dataset_config(
        **cfg.train_dataset
    ),
    lazy=cfg.dataloader.lazy,
    keep_files_open=cfg.dataloader.keep_files_open,
    
)
train_sampling_intervals = train_dataset.get_sampling_intervals()

In [4]:
from torch_brain.models import POYO
from torch_brain.registry import MODALITY_REGISTRY
import torch
import numpy as np
from typing import Any, Dict

device= "cuda" if torch.cuda.is_available() else "cpu"
poyo_model = POYO(
    sequence_length=1.0,
    readout_spec=MODALITY_REGISTRY['cursor_velocity_2d'],
    latent_step=1.0 / 8,
    num_latents_per_step=16,
    dim=64,
    depth=6,
    dim_head=64,
    cross_heads=2,
    self_heads=8,
).to(device)
poyo_model.unit_emb.initialize_vocab(train_dataset.get_unit_ids())
poyo_model.session_emb.initialize_vocab(train_dataset.get_session_ids())

/cs/student/projects1/ml/2024/mlaimon/anaconda3/envs/foundational_ssm/lib/python3.13/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
import time
from tqdm import tqdm
from torch_brain.data import Dataset, collate, chain
from torch_brain.data.sampler import RandomFixedWindowSampler

from foundational_ssm.constants import DATA_ROOT
from foundational_ssm.data_utils.dataset import TorchBrainDataset
from foundational_ssm.data_utils.loaders import get_dataset_config
from torch_brain.models import POYO
from torch_brain.registry import MODALITY_REGISTRY

train_sampler = RandomFixedWindowSampler(
    sampling_intervals=train_sampling_intervals,
    window_length=1.0,
    # batch_size=256,
)
train_loader = DataLoader(
    dataset=train_dataset,
    sampler=train_sampler,
    collate_fn=collate,
    num_workers=16,
    pin_memory=True,
    batch_size=1024,
)
train_dataset.transform = poyo_model.tokenize

poyo_tokenize_time = np.zeros(1000)
start_time = time.time()
for i, batch in tqdm(enumerate(train_loader)):
    poyo_tokenize_time[i] = time.time() - start_time
    start_time = time.time()
poyo_tokenize_time = poyo_tokenize_time[poyo_tokenize_time != 0]
#print the average time per batch
print(f"Average time per batch: {np.mean(poyo_tokenize_time)} seconds")

152it [00:32,  4.72it/s]

Average time per batch: 0.23600583327443977 seconds


In [32]:
train_dataset.get_recording_data('perich_miller_population_2018/c_20131003_center_out_reaching').movement_phases.slice(0,100)

RuntimeError: Unable to synchronously get dataspace (identifier is not of specified type)

In [27]:
import copy
from temporaldata import Interval
copy.copy(train_dataset.get_recording_data('perich_miller_population_2018/c_20131003_center_out_reaching')._domain & Interval(0, 1))

Interval(
  start=[1],
  end=[1]
)

In [ ]:
train_dataset[idx]

RuntimeError: Unable to synchronously get dataspace (identifier is not of specified type)

In [ ]:
train_dataset[idx]

RuntimeError: Unable to synchronously get dataspace (identifier is not of specified type)

In [ ]:
train_dataset[idx]

RuntimeError: Unable to synchronously get dataspace (identifier is not of specified type)

In [7]:
# Profile the get method
profiler = LineProfiler()
profiler.add_function(train_dataset.get)
profiler.add_function(train_dataset._get_data_object)
profiler.add_function(train_dataset._update_data_with_prefixed_ids)
profiler.add_function(train_dataset.transform)

# Run profiling
profiler.enable_by_count()
for i in range(10):  # Profile first 10 calls
    sample = train_dataset.get("perich_miller_population_2018/c_20131003_center_out_reaching", 0.0, 1.0)
profiler.disable_by_count()

# Print results
profiler.print_stats()

Timer unit: 1e-09 s

Total time: 0.0191875 s
File: /cs/student/projects1/ml/2024/mlaimon/foundational_ssm/src/foundational_ssm/data_utils/dataset.py
Function: get at line 315

Line #      Hits         Time  Per Hit   % Time  Line Contents
   315                                               def get(self, recording_id: str, start: float, end: float):
   316                                                   r"""This is the main method to extract a slice from a recording. It returns a
   317                                                   Data object that contains all data for recording :obj:`recording_id` between
   318                                                   times :obj:`start` and :obj:`end`.
   319                                           
   320                                                   Args:
   321                                                       recording_id: The recording id of the slice. This is usually
   322                                              

In [8]:
from foundational_ssm.data_utils.loaders import transform_brainsets_to_fixed_dim_samples_with_binning_and_smoothing, transform_brainsets_to_fixed_dim_samples


train_sampler = RandomFixedWindowSampler(
    sampling_intervals=train_sampling_intervals,
    window_length=1.0,
    # batch_size=256,
)
train_loader = DataLoader(
    dataset=train_dataset,
    sampler=train_sampler,
    collate_fn=collate,
    num_workers=16,
    pin_memory=True,
    batch_size=1024,
)
train_dataset.transform = transform_brainsets_to_fixed_dim_samples

poyo_tokenize_time = np.zeros(1000)
start_time = time.time()
for i, batch in tqdm(enumerate(train_loader)):
    poyo_tokenize_time[i] = time.time() - start_time
    start_time = time.time()
poyo_tokenize_time = poyo_tokenize_time[poyo_tokenize_time != 0]
#print the average time per batch
print(f"Average time per batch: {np.mean(poyo_tokenize_time)} seconds")

152it [00:28,  5.33it/s]

Average time per batch: 0.21331487831316495 seconds


In [14]:
from foundational_ssm.data_utils.loaders import transform_brainsets_to_fixed_dim_samples_with_binning_and_smoothing, transform_brainsets_to_fixed_dim_samples


train_sampler = RandomFixedWindowSampler(
    sampling_intervals=train_sampling_intervals,
    window_length=1.0,
    # batch_size=256,
)
train_loader = DataLoader(
    dataset=train_dataset,
    sampler=train_sampler,
    collate_fn=collate,
    num_workers=16,
    pin_memory=True,
    batch_size=1024,
)
train_dataset.transform = transform_brainsets_to_fixed_dim_samples_with_binning_and_smoothing

poyo_tokenize_time = np.zeros(1000)
start_time = time.time()
for i, batch in tqdm(enumerate(train_loader)):
    poyo_tokenize_time[i] = time.time() - start_time
    start_time = time.time()

106it [00:42,  2.50it/s]


In [18]:
from foundational_ssm.data_utils.loaders import transform_brainsets_to_fixed_dim_samples_with_binning_and_smoothing, transform_brainsets_to_fixed_dim_samples


train_sampler = RandomFixedWindowSampler(
    sampling_intervals=train_sampling_intervals,
    window_length=1.0,
    # batch_size=256,
)
train_loader = DataLoader(
    dataset=train_dataset,
    sampler=train_sampler,
    collate_fn=collate,
    num_workers=12,
    pin_memory=True,
    batch_size=1024,
)
train_dataset.transform = transform_brainsets_to_fixed_dim_samples_with_binning_and_smoothing

poyo_tokenize_time = np.zeros(1000)
start_time = time.time()
for i, batch in tqdm(enumerate(train_loader)):
    poyo_tokenize_time[i] = time.time() - start_time
    start_time = time.time()

152it [00:53,  2.82it/s]
